In [2]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install --upgrade mxnet-cu100
!pip install autogluon
!pip install -U ipykernel

--2020-05-20 11:09:02--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2571184 (2.5M) [text/plain]
Saving to: ‘doctor_train.csv.1’

doctor_train.csv.1  100%[===================>]   2.45M  --.-KB/s    in 0.1s    

2020-05-20 11:09:03 (19.9 MB/s) - ‘doctor_train.csv.1’ saved [2571184/2571184]

--2020-05-20 11:09:07--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [3]:
import autogluon as ag
from autogluon import TabularPrediction as task
import time
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])
df_D=df_D.fillna(0)
# df_D[df_D < 0] = 0
test_df_D=test_df_D.fillna(0)
# test_df_D[test_df_D < 0] = 0

from imblearn.over_sampling import SMOTE
smt = SMOTE(ratio='auto')
df_D_over, df_Y_over = smt.fit_resample(df_D, df['Y'])
df_D_over = pd.DataFrame(df_D_over)
test_df_D.columns = df_D_over.columns
df_D_over['Y'] = df_Y_over

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/imblearn/utils/deprecation.py:53: DeprecationWarning: 'ratio' is deprecated from 0.4 and will be removed in 0.6 for the estimator <class 'imblearn.over_sampling._smote.SMOTE'>. Use 'sampling_strategy' instead.
  category=

In [4]:
predictor = task.fit(train_data=df_D_over, 
                     label='Y', 
                     eval_metric='accuracy',
                     auto_stack=True,
                     verbosity=2)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200520_111039/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200520_111039/
Train Data Rows:    49774
Train Data Columns: 52
Preprocessing data ...
Here are the first 10 unique label values in your data:  [0 1]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Feature Generator processed 49774 data points with 51 features
Original Features:
	float features: 51
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 51
	object features: 0
	int features: 0
	Data preprocessing and feature engineering runtime = 0.33s ...
AutoGluon will gauge predictive performance using evaluatio

In [5]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                    model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0                 weighted_ensemble_k0_l2   0.972797      26.595787  2764.873875                0.022530          14.919418            2       True
1           CatboostClassifier_STACKER_l1   0.972777      23.398867  2663.710222                0.133157          40.912792            1       True
2   RandomForestClassifierGini_STACKER_l1   0.970246      25.440383  2779.195353                2.174673         156.397923            1       True
3   RandomForestClassifierEntr_STACKER_l1   0.970025      25.441880  2785.858618                2.176170         163.061189            1       True
4     ExtraTreesClassifierEntr_STACKER_l1   0.967533      26.440100  2709.041666                3.174390          86.244236            1       True
5     ExtraTreesClassifierGini_STACKER_l1   0.9666

In [6]:
%%time
y_hat = predictor.predict(test_df_D)

CPU times: user 43.8 s, sys: 9.85 s, total: 53.6 s
Wall time: 3min 46s


In [0]:
le_y = le.inverse_transform(y_hat)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')